# Fine Tune

**Reference:** https://neptune.ai/blog/hugging-face-pre-trained-models-find-the-best

In [1]:
# !pip install -r requirements.txt

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import torch

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = "cpu"

print(device)

mps


## Load Model

In [5]:
model_mbart = "facebook/mbart-large-50-many-to-one-mmt"

model_name = f"mbart-finetuned-cn-to-en-auto"
model_path = f"../models/{model_name}"

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [7]:
sentence = "开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快"

tokenizer.src_lang = 'zh_CN'
tokenizer.tgt_lang = 'en_XX'

encoded = tokenizer(sentence, return_tensors="pt").to(device)
model = model.to(device)
generated_tokens = model.generate(**encoded)
decoded = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

print(decoded)

In the case of turning on the air conditioner, the electric range drops too fast, especially when the weather is cold in winter, not turning on the air conditioner is not possible, the weather freezes, and the electric range drops faster.


## Preprocess Data

### Load Data

In [6]:
import pandas as pd
from datasets import Dataset

In [7]:
df_1 = pd.read_excel('../data/trans/2023_NCVQS_text.xlsx', sheet_name=0)
df_1.rename(columns = {'Detail complains breakdown  (Chinese)': 'Chinese', 'Translation': 'English'}, inplace=True)

df_2 = pd.read_excel('../data/trans/2023_NCVQS_text.xlsx', sheet_name=1)

zh_columns = df_2.columns[:4]
en_columns = df_2.columns[4:]

df_2['Chinese'] = df_2[zh_columns].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
df_2['English'] = df_2[en_columns].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

df_2.drop(columns=en_columns, inplace=True)
df_2.drop(columns=zh_columns, inplace=True)

df = pd.concat([df_1, df_2], axis = 0)
df = df.dropna()
df.drop_duplicates(keep='first', subset='Chinese', inplace=True)

In [8]:
print("Length: ", len(df), "\n Num of NaN: ", df.isna().sum().sum())

Length:  447 
 Num of NaN:  0


In [9]:
df.head()

,Chinese,English
0,开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快,"In the case of turning on the air conditioner,..."
1,车机流畅度差，容易卡死机，车机系统，启动载入很慢，换挡杆前的车机，使用任何功能都有概率死机，...,"The smoothness of the IHU is poor, easy to jam..."
2,整车的悬架系统，在过减速带时，速度在20码以下，但是车身的抖动还是很厉害，舒适性为第一的，美...,"The suspension system of the whole car, when c..."
3,大众车的通病，车子的隔音效果不太理想，车速在90码以上，车内的胎噪声就很明显了，必须把音量调...,"The common problem of Volkswagen, the sound in..."
4,车辆外观很不错，但是车标在晚上不能发亮，要是可以发亮的话会更拉风一点,"The appearance of the vehicle is very good, bu..."


In [10]:
tmp = df.copy()

### Transform Data

In [11]:
df = df.assign(translation=df.apply(lambda row:{'zh': row['Chinese'], 'en': row['English']}, axis=1))
df.reset_index(inplace=True)
df.drop(['index', 'Chinese', 'English'], axis=1, inplace=True)

In [12]:
df.head()

,translation
0,{'zh': '开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一...
1,{'zh': '车机流畅度差，容易卡死机，车机系统，启动载入很慢，换挡杆前的车机，使用任何功...
2,{'zh': '整车的悬架系统，在过减速带时，速度在20码以下，但是车身的抖动还是很厉害，舒...
3,{'zh': '大众车的通病，车子的隔音效果不太理想，车速在90码以上，车内的胎噪声就很明显...
4,"{'zh': '车辆外观很不错，但是车标在晚上不能发亮，要是可以发亮的话会更拉风一点', '..."


In [13]:
df.iloc[0][0]

{'zh': '开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快',
 'en': "In the case of turning on the air conditioner, the electric range drops too fast, especially when the weather is cold in winter, if you don't turn on the air conditioner, the weather freezes, the battery life will fall faster."}

In [14]:
print("Dimension: ", df.shape)

Dimension:  (447, 1)


In [15]:
dataset = Dataset.from_pandas(df, split='train')
dataset = dataset.train_test_split(test_size=0.05)

In [16]:
print(type(dataset['train']))
print(dataset['train'][:1])

<class 'datasets.arrow_dataset.Dataset'>
{'translation': [{'en': 'The body is magnificent, the functions is high, and the acceleration is fast. Good quality, fast price reduction, small third row. High functions, fast acceleration. Does not preserve value, the third row is small.', 'zh': '车身大气，配置高，加速快 质量不错，降价快，第三排小 配置高，加速快 不保值，第三排小'}]}


In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 424
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 23
    })
})

### Tokenize Datasets

In [18]:
prefix = "" #for mBART and MarianMT
max_input_length = 512
max_target_length = 512

source_lang = "zh"
target_lang = "en"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
   
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/424 [00:00<?, ? examples/s]

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

## Train and Fine-tune the Model

### Model Setup

In [19]:
from transformers import Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer
import evaluate

In [20]:
from datetime import datetime

# Get the current date
today = datetime.now()

# Format the date as "DDMMYYYY"
formatted_date = today.strftime("%d%m%Y")

print(formatted_date)

09022024


In [21]:
model_name = f"mbart-finetuned-cn-to-en-auto"
model_path = f"../models/{model_name}"

batch_size = 4

args = Seq2SeqTrainingArguments(
   output_dir=model_path,
   evaluation_strategy="epoch",
   learning_rate=2e-5,
   per_device_train_batch_size=batch_size,
   per_device_eval_batch_size=batch_size,
   weight_decay=0.01,
   save_total_limit=3,
   num_train_epochs=2,
   predict_with_generate=True,
)

In [22]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) # default setting

In [23]:
import numpy as np
import evaluate

metric = evaluate.load("sacrebleu")
meteor = evaluate.load('meteor')

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    meteor_result = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result = {'bleu' : result['score']}
    result["gen_len"] = np.mean(prediction_lens)
    result["meteor"] = meteor_result["meteor"]
    result = {k: round(v, 4) for k, v in result.items()}
    return result

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [24]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

### Train and Save the Model

In [25]:
trainer.train()

You're using a MBart50TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len,Meteor
1,No log,0.641811,45.133400,52.478300,0.738700
2,No log,0.632328,47.199900,53.478300,0.752000


TrainOutput(global_step=212, training_loss=0.5843993852723319, metrics={'train_runtime': 149.5947, 'train_samples_per_second': 5.669, 'train_steps_per_second': 1.417, 'total_flos': 109084508553216.0, 'train_loss': 0.5843993852723319, 'epoch': 2.0})

In [26]:
eval_result = trainer.evaluate(tokenized_datasets['test'])

In [27]:
eval_result

{'eval_loss': 0.6323279142379761,
 'eval_bleu': 47.1999,
 'eval_gen_len': 53.4783,
 'eval_meteor': 0.752,
 'eval_runtime': 9.9321,
 'eval_samples_per_second': 2.316,
 'eval_steps_per_second': 0.604,
 'epoch': 2.0}

In [28]:
trainer.save_model()

In [29]:
# trainer.push_to_hub()

### Test

In [30]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [31]:
sentence = "开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快"

tokenizer.src_lang = 'zh_CN'
tokenizer.tgt_lang = 'en_XX'

encoded = tokenizer(sentence, return_tensors="pt").to(device)
model = model.to(device)
generated_tokens = model.generate(**encoded)
decoded = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

print(decoded)

 If you turn on the air conditioner, the electric range is too fast to fall off, especially in winter when the weather is cold and cold, it is not possible to not turn on the air conditioner without turning on the air conditioner, as the weather freezes and freezes, the electric range will fall off faster.


> "开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快"

> 'If the air conditioner is on, the flight resumes too quickly, especially in winter when the weather is cold. If the air conditioner is not on, the flight resumes faster as soon as the weather is cold'

In [32]:
def translate(sentence):
    encoded = tokenizer(sentence, return_tensors="pt").to(device)
    generated_tokens = model.generate(**encoded)
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

In [33]:
for i in dataset['test']['translation'][:5]:
    print("="*30)
    print(i['zh'])
    print(i['en'])
    print("="*10)
    result = translate(i['zh'])
    print(result)

纯电里程较少 质量还可以 操控感比较好，纯电模式下比较经济 电池里程短，后备箱不够大，后排座椅希望更大空间，车内座椅下布线太散乱
Pure electric mileage is less. The quality is ok. The control feeling is better, and the pure electric mode is more economical. The battery mileage is short, the trunk is not large enough, the rear seats want more space, and the wiring under the seats in the car is too loose.
The range of pure electricity is relatively small. The quality is good. The control feeling is relatively good, and the pure electric mode is relatively economical under pure electric mode. The battery range is short, the trunk is not large enough, the rear seat seat hopes for more space, the back seat seat hopes for more space, the underwiring of the seat in the car is too scattered and cluttered.
前排空间大，加速快，刹车灵敏，驾驶舒适 没有质量的问题 前排空间大，提速快，刹车灵敏 后排座椅靠背软一些，就更舒适了
Large front space, fast acceleration, sensitive brakes, comfortable driving. No quality issues. Large front space, fast acceleration, sensitive brakes. The rear seat backrests are softer and more c